# IMDB Investigation

### Import Library

In [8]:
# import
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook


### Import Data

In [9]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

basics = pd.read_csv(basics_url, low_memory=False, sep='\t')
akas = pd.read_csv(akas_url,  low_memory=False, sep='\t')
ratings = pd.read_csv(ratings_url, low_memory=False, sep='\t')

### Review Dataframes

In [10]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [11]:
#creating backup of 'basics' dataframe
df_b_save = basics.copy()

In [12]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [13]:
df_a_save = akas.copy()

In [14]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1984
1,tt0000002,5.8,265
2,tt0000003,6.5,1840
3,tt0000004,5.5,178
4,tt0000005,6.2,2623


In [15]:
df_r_save = ratings.copy()

### Preprocessing Cleanup

#### title.basics

##### Find NAN Values

In [16]:
basics = basics.replace({'\\N':np.nan})

##### Evaluating NAN values in columns

In [17]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1345133
endYear           9876330
runtimeMinutes    7020913
genres             449282
dtype: int64

In [18]:
#Display the percentage of missing values by row. 
print(basics.isna().sum()/len(basics)*100)

tconst             0.000000
titleType          0.000000
primaryTitle       0.000110
originalTitle      0.000110
isAdult            0.000010
startYear         13.470299
endYear           98.902575
runtimeMinutes    70.308138
genres             4.499156
dtype: float64


##### Dropping column endYear since this primarily an investigation of movies

In [19]:
basics.drop(['endYear'], axis=1, inplace=True)

##### Eliminate movies that are null for runtimeMinutes

In [20]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

##### Eliminate movies that are null for genre

In [21]:
basics.dropna(subset=['genres'], inplace=True)

##### keep only titleType==movie

In [22]:
basics = basics[basics['titleType'] == 'movie']

##### Eliminating NAN values for 'startYear'

In [23]:
basics.dropna(subset=['startYear'], inplace=True)

##### NAN values resolved

In [24]:
print(basics.isna().sum()/len(basics)*100)

tconst            0.0
titleType         0.0
primaryTitle      0.0
originalTitle     0.0
isAdult           0.0
startYear         0.0
runtimeMinutes    0.0
genres            0.0
dtype: float64


##### Modifying Datatypes to conform to data dictionary

In [25]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int","runtimeMinutes":"int", "genres":"string"})

In [26]:
basics.dtypes

tconst            string
titleType         string
primaryTitle      string
originalTitle     string
isAdult            int32
startYear          int32
runtimeMinutes     int32
genres            string
dtype: object

##### keep startYear 2000-2022

In [27]:
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

##### Eliminate movies that include "Documentary" in genre

In [28]:
basics = basics[~basics['genres'].str.contains(r'documentary', case=False)]


#### title.akas

##### Find NAN Values

In [29]:
akas =akas.replace({'\\N':np.nan})

##### Filtering out all non US Films

In [30]:
akas = akas[akas['region'] == 'US']

##### Evaluating NAN values in columns

In [31]:
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1445485
types               468495
attributes         1402539
isOriginalTitle       1342
dtype: int64

In [32]:
print(akas.isna().sum()/len(akas)*100)

titleId             0.000000
ordering            0.000000
title               0.000000
region              0.000000
language           99.725210
types              32.321859
attributes         96.762329
isOriginalTitle     0.092586
dtype: float64


##### Dropping Columns Language and Attributes due to lack of data, as well as isOriginalTitle since it doesnt directly relate to investigation

In [33]:
akas.drop(['language','attributes','isOriginalTitle'], axis=1, inplace=True)

##### Replacing NAN values with Unknown for types column

In [34]:
type_of = 'unknown'
akas['types'].fillna(value = type_of, inplace = True)

##### NAN Values Resolved

In [35]:
akas.isna().sum()

titleId     0
ordering    0
title       0
region      0
types       0
dtype: int64

##### Modifying Datatypes to conform to Data Dictionary

In [36]:
akas = akas.astype({"titleId":"string","title":"string","region":"string","types":"string"})

In [37]:
akas.dtypes

titleId     string
ordering     int64
title       string
region      string
types       string
dtype: object

#### title.ratings

##### Find NAN Values

In [38]:
ratings = ratings.replace({'\\N':np.nan})

In [39]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

* No NAN values found

##### Modifying Datatypes to conform to data dictionary

In [40]:
ratings = ratings.astype({"tconst":"string"})

In [41]:
ratings.dtypes

tconst            string
averageRating    float64
numVotes           int64
dtype: object

#### Filtering for Title Basics with akas dataframe for US only

In [42]:
keepers1 =basics['tconst'].isin(akas['titleId'])
keepers1

34803       True
61115       True
67667       True
86794       True
93931       True
           ...  
9985591     True
9985600     True
9985639    False
9985684     True
9985768    False
Name: tconst, Length: 147879, dtype: bool

In [43]:
basics = basics[keepers1]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86794,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
93931,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama
...,...,...,...,...,...,...,...,...
9985056,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
9985451,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
9985591,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
9985600,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


#### Filtering for ratings with akas dataframe for US only

In [44]:
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1327273    False
1327274    False
1327275    False
1327276    False
1327277    False
Name: tconst, Length: 1327278, dtype: bool

In [45]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1984
1,tt0000002,5.8,265
4,tt0000005,6.2,2623
5,tt0000006,5.1,182
6,tt0000007,5.4,820
...,...,...,...
1327239,tt9916200,8.1,230
1327240,tt9916204,8.2,264
1327247,tt9916348,8.3,18
1327248,tt9916362,6.4,5405


### Saving modified dataframes to file for later use

In [46]:
basics.to_csv("Data/title_basics1.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas1.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings1.csv.gz",compression='gzip',index=False)

### Project 3 - Part 2 Extract from TMDB (Core)

### Loading Files

In [47]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas1.csv.gz',
 'title_basics1.csv.gz',
 'title_ratings1.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [48]:
# Load API credentials from JSON file
#with open('/Users/Paul Lipska/.secret/tmdb_api.json', 'r') as f:
with open('/Users/paull/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)

# Set API key
tmdb.API_KEY = login['api-key']

# Create a movie object using the Movies function from tmdb
movie = tmdb.Movies(603)

# Retrieve movie information
info = movie.info()

### Loading df from previous exercise

In [49]:
# Load in the dataframe from project part 1
basics = pd.read_csv('Data/title_basics1.csv.gz')

In [50]:
YEARS_TO_GET = [2000,2001]

In [55]:
errors = []

In [51]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama


In [52]:
#get movie certification
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movei object fro the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .release dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] == 'US':
            ## save a 'certification' key in info with the certification
            info['certification'] = c['certification']
            
    return info

In [53]:
#append function
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [56]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    #if it does not exist: create it
    if file_exists == False:
    #save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    #saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    
    #saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    movie_ids

    #Load existing data from json into a dataframe called 'previous_df'
    previous_df = pd.read_json(JSON_FILE)

    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])



    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/214 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/240 [00:00<?, ?it/s]